In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
data_path = '/home/korawich/Desktop/AutoML/Dataset/DDMut_data/'
gbsig_path = '/home/korawich/Desktop/AutoML/Dataset/DDMut_data/Graph-based/'
final_path = '/home/korawich/Desktop/AutoML/Dataset/DDMut_data/Final/'

In [3]:
#Input
train_other_path  = data_path + 'train9028_label_feat 1.xlsx'
test_1_other_path = data_path + 'test552_label_feat.xlsx'
test_2_other_path = data_path + 'test1304_label_feat.xlsx'
test_3_other_path = data_path + 'test2024_label_feat.xlsx'
test_4_other_path = data_path + 'tpmt_label_feat.xlsx'
test_5_other_path = data_path + 'pten_label_feat.xlsx'

train_gbsig_path = gbsig_path + 'train_output.txt'
test_1_gbsig_path = gbsig_path + 'test552_output.txt'
test_2_gbsig_path = gbsig_path + 'test1304_output.txt'
test_3_gbsig_path = gbsig_path + 'test2024_output.txt'
test_4_gbsig_path = gbsig_path + 'tpmt_output.txt'
test_5_gbsig_path = gbsig_path + 'pten_output.txt'

#Output
train_info_path  = final_path + 'train_info.csv'
test_1_info_path = final_path + 'test_1_info.csv'
test_2_info_path = final_path + 'test_2_info.csv'
test_3_info_path = final_path + 'test_3_info.csv'
test_4_info_path = final_path + 'test_4_info.csv'
test_5_info_path = final_path + 'test_5_info.csv'

train_X_path  = final_path + 'train_X.csv'
test_1_X_path = final_path + 'test_1_X.csv'
test_2_X_path = final_path + 'test_2_X.csv'
test_3_X_path = final_path + 'test_3_X.csv'
test_4_X_path = final_path + 'test_4_X.csv'
test_5_X_path = final_path + 'test_5_X.csv'

train_y_path  = final_path + 'train_y.csv'
test_1_y_path = final_path + 'test_1_y.csv'
test_2_y_path = final_path + 'test_2_y.csv'
test_3_y_path = final_path + 'test_3_y.csv'
test_4_y_path = final_path + 'test_4_y.csv'
test_5_y_path = final_path + 'test_5_y.csv'

train_y_cls_path  = final_path + 'train_y_cls.csv'
test_1_y_cls_path = final_path + 'test_1_y_cls.csv'
test_2_y_cls_path = final_path + 'test_2_y_cls.csv'
test_3_y_cls_path = final_path + 'test_3_y_cls.csv'
test_4_y_cls_path = final_path + 'test_4_y_cls.csv'
test_5_y_cls_path = final_path + 'test_5_y_cls.csv'

In [4]:
#Create a list of gbsig feat in other(all) features to remove them and then add back from GBSIG files
test_2_other_df = pd.read_excel(test_2_other_path)
test_2_gbsig_df = pd.read_csv(test_2_gbsig_path, delimiter=',')

list_ignore_temp = []
for i in test_2_other_df.columns:
    if i in test_2_gbsig_df.columns:
        if i != 'pdb_mut_chain':
            list_ignore_temp.append(i)

In [5]:
ignore_for_comp_feat = np.array(['dynamut2',
                        'deformation_calpha',
                        'deformation_anm',
                        'deformation_pfanm',
                        'deformation_reach',
                        'deformation_sdenm',
                        'fluctuation_calpha',
                        'fluctuation_anm',
                        'fluctuation_pfanm',
                        'fluctuation_reach',
                        'fluctuation_sdenm',
                        'mCSM_ddG'])

In [25]:
ignore_for_3_4_dataset_comp_feat = ['protein',
                                    'chain',
                                    'mutation',
                                    'exp_score',
                                    'Clash',
                                    'Halogen',
                                    'Metal',
                                    'd_Clash',
                                    'd_Halogen',
                                    'd_Metal',
                                    'Hydro',
                                    'Pos',
                                    'Neg',
                                    'Acc',
                                    'Don',
                                    'Aro',
                                    'Sul',
                                    'Neut']

In [6]:
def generate_data(other_path, gbsig_path, info_path, X_path, y_path, y_cls_path):
    gbsig_df = pd.read_csv(gbsig_path, delimiter=',').drop('pdb_mut_chain', axis=1)
    other_df = pd.read_excel(other_path).drop(list_ignore_temp, axis=1)

    other_df = other_df[other_df.columns.difference(list_ignore_temp)]
    other_df = other_df[other_df.columns.difference(ignore_for_comp_feat)]
    
    print('other_df.shape = ', other_df.shape)
    print('gbsig_df.shape = ', gbsig_df.shape)
    
    final_df = pd.concat([other_df, gbsig_df], axis=1)

    print('final_df.shape = ', final_df.shape)
    
    #Generate - X, y_reg, y_class
    info_col = ['pdb_mut_chain', 'uniprot_mut', 'pH', 'TEMP', 'type']
    info_data_df = final_df[info_col]
    y_arr = final_df['ddG']
    X_arr = final_df.drop(['ddG', 'pdb_mut_chain', 'uniprot_mut', 'pH', 'TEMP', 'type'], axis=1)
    y_arr_cls = np.where(y_arr < 0, 0, np.where(y_arr > 0, 1, 0))
    
    info_data_df.to_csv(info_path, index=True)
    pd.DataFrame(X_arr).to_csv(X_path, index=True)
    pd.DataFrame(y_arr).to_csv(y_path, index=True)
    pd.DataFrame(y_arr_cls).to_csv(y_cls_path, index=True)

In [7]:
def generate_data2(other_path, gbsig_path, info_path, X_path, y_path, y_cls_path):
    gbsig_df = pd.read_csv(gbsig_path, delimiter=',').drop('pdb_mut_chain', axis=1)
    other_df = pd.read_excel(other_path)

    other_df = other_df[other_df.columns.difference(list_ignore_temp)]
    other_df = other_df[other_df.columns.difference(ignore_for_comp_feat)]

    print('other_df.shape = ', other_df.shape)
    print('gbsig_df.shape = ', gbsig_df.shape)
    
    final_df = pd.concat([other_df, gbsig_df], axis=1)

    print('final_df.shape = ', final_df.shape)
    
    #Generate - X, y_reg, y_class
    info_col = ['pdb_mut_chain', 'uniprot_mut', 'pH', 'TEMP', 'type']
    info_data_df = final_df[info_col]
    y_arr = final_df['ddG']
    X_arr = final_df.drop(['ddG', 'pdb_mut_chain', 'uniprot_mut', 'pH', 'TEMP', 'type'], axis=1)
    y_arr_cls = np.where(y_arr < 0, 0, np.where(y_arr > 0, 1, 0))
    
    info_data_df.to_csv(info_path, index=True)
    pd.DataFrame(X_arr).to_csv(X_path, index=True)
    pd.DataFrame(y_arr).to_csv(y_path, index=True)
    pd.DataFrame(y_arr_cls).to_csv(y_cls_path, index=True)

In [24]:
def generate_data_3(other_path, gbsig_path, info_path, X_path, y_path, y_cls_path):
    gbsig_df = pd.read_csv(gbsig_path, delimiter=',').drop('pdb_mut_chain', axis=1)
    other_df = pd.read_excel(other_path).drop(list_ignore_temp, axis=1)

    other_df = other_df[other_df.columns.difference(ignore_for_3_4_dataset_comp_feat)]
    
    print('other_df.shape = ', other_df.shape)
    print('gbsig_df.shape = ', gbsig_df.shape)
    
    final_df = pd.concat([other_df, gbsig_df], axis=1)

    print('final_df.shape = ', final_df.shape)
    
    #Generate - X, y_reg, y_class
    info_col = ['pdb_mut_chain', 'protein', 'chain', 'mutation', 'exp_score']
    info_data_df = final_df[info_col]
    X_arr = final_df.drop(['pdb_mut_chain', 'protein', 'chain', 'mutation', 'exp_score'], axis=1)

    info_data_df.to_csv(info_path, index=True)
    pd.DataFrame(X_arr).to_csv(X_path, index=True)

In [9]:
generate_data(train_other_path, train_gbsig_path, train_info_path, train_X_path, train_y_path, train_y_cls_path)
generate_data2(test_1_other_path, test_1_gbsig_path, test_1_info_path, test_1_X_path, test_1_y_path, test_1_y_cls_path)
generate_data(test_2_other_path, test_2_gbsig_path, test_2_info_path, test_2_X_path, test_2_y_path, test_2_y_cls_path)
generate_data(test_3_other_path, test_3_gbsig_path, test_3_info_path, test_3_X_path, test_3_y_path, test_3_y_cls_path)

other_df.shape =  (9028, 204)
gbsig_df.shape =  (9028, 692)
final_df.shape =  (9028, 896)
other_df.shape =  (552, 204)
gbsig_df.shape =  (552, 692)
final_df.shape =  (552, 896)
other_df.shape =  (1304, 204)
gbsig_df.shape =  (1304, 692)
final_df.shape =  (1304, 896)
other_df.shape =  (2024, 204)
gbsig_df.shape =  (2024, 692)
final_df.shape =  (2024, 896)


generate_data_3(test_4_other_path, test_4_gbsig_path, test_4_info_path, test_4_X_path, test_4_y_path, test_4_y_cls_path)
generate_data_3(test_5_other_path, test_5_gbsig_path, test_5_info_path, test_5_X_path, test_5_y_path, test_5_y_cls_path)

In [49]:
gbsig_feat = np.array(['Acc:Acc-10.00', 'Acc:Aro-10.00', 'Acc:Don-10.00',
       'Acc:Hydro-10.00', 'Acc:Neg-10.00', 'Acc:Neutral-10.00',
       'Acc:Pos-10.00', 'Acc:Sul-10.00', 'Aro:Aro-10.00', 'Aro:Don-10.00',
       'Aro:Hydro-10.00', 'Aro:Neg-10.00', 'Aro:Neutral-10.00',
       'Aro:Pos-10.00', 'Aro:Sul-10.00', 'Don:Don-10.00',
       'Don:Hydro-10.00', 'Don:Neg-10.00', 'Don:Neutral-10.00',
       'Don:Pos-10.00', 'Don:Sul-10.00', 'Hydro:Hydro-10.00',
       'Hydro:Neg-10.00', 'Hydro:Neutral-10.00', 'Hydro:Pos-10.00',
       'Hydro:Sul-10.00', 'Neg:Neg-10.00', 'Neg:Neutral-10.00',
       'Neg:Pos-10.00', 'Neg:Sul-10.00', 'Neutral:Neutral-10.00',
       'Neutral:Pos-10.00', 'Neutral:Sul-10.00', 'Pos:Pos-10.00',
       'Pos:Sul-10.00', 'Sul:Sul-10.00', 'Acc:Acc-9.50', 'Acc:Aro-9.50',
       'Acc:Don-9.50', 'Acc:Hydro-9.50', 'Acc:Neg-9.50',
       'Acc:Neutral-9.50', 'Acc:Pos-9.50', 'Acc:Sul-9.50', 'Aro:Aro-9.50',
       'Aro:Don-9.50', 'Aro:Hydro-9.50', 'Aro:Neg-9.50',
       'Aro:Neutral-9.50', 'Aro:Pos-9.50', 'Aro:Sul-9.50', 'Don:Don-9.50',
       'Don:Hydro-9.50', 'Don:Neg-9.50', 'Don:Neutral-9.50',
       'Don:Pos-9.50', 'Don:Sul-9.50', 'Hydro:Hydro-9.50',
       'Hydro:Neg-9.50', 'Hydro:Neutral-9.50', 'Hydro:Pos-9.50',
       'Hydro:Sul-9.50', 'Neg:Neg-9.50', 'Neg:Neutral-9.50',
       'Neg:Pos-9.50', 'Neg:Sul-9.50', 'Neutral:Neutral-9.50',
       'Neutral:Pos-9.50', 'Neutral:Sul-9.50', 'Pos:Pos-9.50',
       'Pos:Sul-9.50', 'Sul:Sul-9.50', 'Acc:Acc-9.00', 'Acc:Aro-9.00',
       'Acc:Don-9.00', 'Acc:Hydro-9.00', 'Acc:Neg-9.00',
       'Acc:Neutral-9.00', 'Acc:Pos-9.00', 'Acc:Sul-9.00', 'Aro:Aro-9.00',
       'Aro:Don-9.00', 'Aro:Hydro-9.00', 'Aro:Neg-9.00',
       'Aro:Neutral-9.00', 'Aro:Pos-9.00', 'Aro:Sul-9.00', 'Don:Don-9.00',
       'Don:Hydro-9.00', 'Don:Neg-9.00', 'Don:Neutral-9.00',
       'Don:Pos-9.00', 'Don:Sul-9.00', 'Hydro:Hydro-9.00',
       'Hydro:Neg-9.00', 'Hydro:Neutral-9.00', 'Hydro:Pos-9.00',
       'Hydro:Sul-9.00', 'Neg:Neg-9.00', 'Neg:Neutral-9.00',
       'Neg:Pos-9.00', 'Neg:Sul-9.00', 'Neutral:Neutral-9.00',
       'Neutral:Pos-9.00', 'Neutral:Sul-9.00', 'Pos:Pos-9.00',
       'Pos:Sul-9.00', 'Sul:Sul-9.00', 'Acc:Acc-8.50', 'Acc:Aro-8.50',
       'Acc:Don-8.50', 'Acc:Hydro-8.50', 'Acc:Neg-8.50',
       'Acc:Neutral-8.50', 'Acc:Pos-8.50', 'Acc:Sul-8.50', 'Aro:Aro-8.50',
       'Aro:Don-8.50', 'Aro:Hydro-8.50', 'Aro:Neg-8.50',
       'Aro:Neutral-8.50', 'Aro:Pos-8.50', 'Aro:Sul-8.50', 'Don:Don-8.50',
       'Don:Hydro-8.50', 'Don:Neg-8.50', 'Don:Neutral-8.50',
       'Don:Pos-8.50', 'Don:Sul-8.50', 'Hydro:Hydro-8.50',
       'Hydro:Neg-8.50', 'Hydro:Neutral-8.50', 'Hydro:Pos-8.50',
       'Hydro:Sul-8.50', 'Neg:Neg-8.50', 'Neg:Neutral-8.50',
       'Neg:Pos-8.50', 'Neg:Sul-8.50', 'Neutral:Neutral-8.50',
       'Neutral:Pos-8.50', 'Neutral:Sul-8.50', 'Pos:Pos-8.50',
       'Pos:Sul-8.50', 'Sul:Sul-8.50', 'Acc:Acc-8.00', 'Acc:Aro-8.00',
       'Acc:Don-8.00', 'Acc:Hydro-8.00', 'Acc:Neg-8.00',
       'Acc:Neutral-8.00', 'Acc:Pos-8.00', 'Acc:Sul-8.00', 'Aro:Aro-8.00',
       'Aro:Don-8.00', 'Aro:Hydro-8.00', 'Aro:Neg-8.00',
       'Aro:Neutral-8.00', 'Aro:Pos-8.00', 'Aro:Sul-8.00', 'Don:Don-8.00',
       'Don:Hydro-8.00', 'Don:Neg-8.00', 'Don:Neutral-8.00',
       'Don:Pos-8.00', 'Don:Sul-8.00', 'Hydro:Hydro-8.00',
       'Hydro:Neg-8.00', 'Hydro:Neutral-8.00', 'Hydro:Pos-8.00',
       'Hydro:Sul-8.00', 'Neg:Neg-8.00', 'Neg:Neutral-8.00',
       'Neg:Pos-8.00', 'Neg:Sul-8.00', 'Neutral:Neutral-8.00',
       'Neutral:Pos-8.00', 'Neutral:Sul-8.00', 'Pos:Pos-8.00',
       'Pos:Sul-8.00', 'Sul:Sul-8.00', 'Acc:Acc-7.50', 'Acc:Aro-7.50',
       'Acc:Don-7.50', 'Acc:Hydro-7.50', 'Acc:Neg-7.50',
       'Acc:Neutral-7.50', 'Acc:Pos-7.50', 'Acc:Sul-7.50', 'Aro:Aro-7.50',
       'Aro:Don-7.50', 'Aro:Hydro-7.50', 'Aro:Neg-7.50',
       'Aro:Neutral-7.50', 'Aro:Pos-7.50', 'Aro:Sul-7.50', 'Don:Don-7.50',
       'Don:Hydro-7.50', 'Don:Neg-7.50', 'Don:Neutral-7.50',
       'Don:Pos-7.50', 'Don:Sul-7.50', 'Hydro:Hydro-7.50',
       'Hydro:Neg-7.50', 'Hydro:Neutral-7.50', 'Hydro:Pos-7.50',
       'Hydro:Sul-7.50', 'Neg:Neg-7.50', 'Neg:Neutral-7.50',
       'Neg:Pos-7.50', 'Neg:Sul-7.50', 'Neutral:Neutral-7.50',
       'Neutral:Pos-7.50', 'Neutral:Sul-7.50', 'Pos:Pos-7.50',
       'Pos:Sul-7.50', 'Sul:Sul-7.50', 'Acc:Acc-7.00', 'Acc:Aro-7.00',
       'Acc:Don-7.00', 'Acc:Hydro-7.00', 'Acc:Neg-7.00',
       'Acc:Neutral-7.00', 'Acc:Pos-7.00', 'Acc:Sul-7.00', 'Aro:Aro-7.00',
       'Aro:Don-7.00', 'Aro:Hydro-7.00', 'Aro:Neg-7.00',
       'Aro:Neutral-7.00', 'Aro:Pos-7.00', 'Aro:Sul-7.00', 'Don:Don-7.00',
       'Don:Hydro-7.00', 'Don:Neg-7.00', 'Don:Neutral-7.00',
       'Don:Pos-7.00', 'Don:Sul-7.00', 'Hydro:Hydro-7.00',
       'Hydro:Neg-7.00', 'Hydro:Neutral-7.00', 'Hydro:Pos-7.00',
       'Hydro:Sul-7.00', 'Neg:Neg-7.00', 'Neg:Neutral-7.00',
       'Neg:Pos-7.00', 'Neg:Sul-7.00', 'Neutral:Neutral-7.00',
       'Neutral:Pos-7.00', 'Neutral:Sul-7.00', 'Pos:Pos-7.00',
       'Pos:Sul-7.00', 'Sul:Sul-7.00', 'Acc:Acc-6.50', 'Acc:Aro-6.50',
       'Acc:Don-6.50', 'Acc:Hydro-6.50', 'Acc:Neg-6.50',
       'Acc:Neutral-6.50', 'Acc:Pos-6.50', 'Acc:Sul-6.50', 'Aro:Aro-6.50',
       'Aro:Don-6.50', 'Aro:Hydro-6.50', 'Aro:Neg-6.50',
       'Aro:Neutral-6.50', 'Aro:Pos-6.50', 'Aro:Sul-6.50', 'Don:Don-6.50',
       'Don:Hydro-6.50', 'Don:Neg-6.50', 'Don:Neutral-6.50',
       'Don:Pos-6.50', 'Don:Sul-6.50', 'Hydro:Hydro-6.50',
       'Hydro:Neg-6.50', 'Hydro:Neutral-6.50', 'Hydro:Pos-6.50',
       'Hydro:Sul-6.50', 'Neg:Neg-6.50', 'Neg:Neutral-6.50',
       'Neg:Pos-6.50', 'Neg:Sul-6.50', 'Neutral:Neutral-6.50',
       'Neutral:Pos-6.50', 'Neutral:Sul-6.50', 'Pos:Pos-6.50',
       'Pos:Sul-6.50', 'Sul:Sul-6.50', 'Acc:Acc-6.00', 'Acc:Aro-6.00',
       'Acc:Don-6.00', 'Acc:Hydro-6.00', 'Acc:Neg-6.00',
       'Acc:Neutral-6.00', 'Acc:Pos-6.00', 'Acc:Sul-6.00', 'Aro:Aro-6.00',
       'Aro:Don-6.00', 'Aro:Hydro-6.00', 'Aro:Neg-6.00',
       'Aro:Neutral-6.00', 'Aro:Pos-6.00', 'Aro:Sul-6.00', 'Don:Don-6.00',
       'Don:Hydro-6.00', 'Don:Neg-6.00', 'Don:Neutral-6.00',
       'Don:Pos-6.00', 'Don:Sul-6.00', 'Hydro:Hydro-6.00',
       'Hydro:Neg-6.00', 'Hydro:Neutral-6.00', 'Hydro:Pos-6.00',
       'Hydro:Sul-6.00', 'Neg:Neg-6.00', 'Neg:Neutral-6.00',
       'Neg:Pos-6.00', 'Neg:Sul-6.00', 'Neutral:Neutral-6.00',
       'Neutral:Pos-6.00', 'Neutral:Sul-6.00', 'Pos:Pos-6.00',
       'Pos:Sul-6.00', 'Sul:Sul-6.00', 'Acc:Acc-5.50', 'Acc:Aro-5.50',
       'Acc:Don-5.50', 'Acc:Hydro-5.50', 'Acc:Neg-5.50',
       'Acc:Neutral-5.50', 'Acc:Pos-5.50', 'Acc:Sul-5.50', 'Aro:Aro-5.50',
       'Aro:Don-5.50', 'Aro:Hydro-5.50', 'Aro:Neg-5.50',
       'Aro:Neutral-5.50', 'Aro:Pos-5.50', 'Aro:Sul-5.50', 'Don:Don-5.50',
       'Don:Hydro-5.50', 'Don:Neg-5.50', 'Don:Neutral-5.50',
       'Don:Pos-5.50', 'Don:Sul-5.50', 'Hydro:Hydro-5.50',
       'Hydro:Neg-5.50', 'Hydro:Neutral-5.50', 'Hydro:Pos-5.50',
       'Hydro:Sul-5.50', 'Neg:Neg-5.50', 'Neg:Neutral-5.50',
       'Neg:Pos-5.50', 'Neg:Sul-5.50', 'Neutral:Neutral-5.50',
       'Neutral:Pos-5.50', 'Neutral:Sul-5.50', 'Pos:Pos-5.50',
       'Pos:Sul-5.50', 'Sul:Sul-5.50', 'Acc:Acc-5.00', 'Acc:Aro-5.00',
       'Acc:Don-5.00', 'Acc:Hydro-5.00', 'Acc:Neg-5.00',
       'Acc:Neutral-5.00', 'Acc:Pos-5.00', 'Acc:Sul-5.00', 'Aro:Aro-5.00',
       'Aro:Don-5.00', 'Aro:Hydro-5.00', 'Aro:Neg-5.00',
       'Aro:Neutral-5.00', 'Aro:Pos-5.00', 'Aro:Sul-5.00', 'Don:Don-5.00',
       'Don:Hydro-5.00', 'Don:Neg-5.00', 'Don:Neutral-5.00',
       'Don:Pos-5.00', 'Don:Sul-5.00', 'Hydro:Hydro-5.00',
       'Hydro:Neg-5.00', 'Hydro:Neutral-5.00', 'Hydro:Pos-5.00',
       'Hydro:Sul-5.00', 'Neg:Neg-5.00', 'Neg:Neutral-5.00',
       'Neg:Pos-5.00', 'Neg:Sul-5.00', 'Neutral:Neutral-5.00',
       'Neutral:Pos-5.00', 'Neutral:Sul-5.00', 'Pos:Pos-5.00',
       'Pos:Sul-5.00', 'Sul:Sul-5.00', 'Acc:Acc-4.50', 'Acc:Aro-4.50',
       'Acc:Don-4.50', 'Acc:Hydro-4.50', 'Acc:Neg-4.50',
       'Acc:Neutral-4.50', 'Acc:Pos-4.50', 'Acc:Sul-4.50', 'Aro:Aro-4.50',
       'Aro:Don-4.50', 'Aro:Hydro-4.50', 'Aro:Neg-4.50',
       'Aro:Neutral-4.50', 'Aro:Pos-4.50', 'Aro:Sul-4.50', 'Don:Don-4.50',
       'Don:Hydro-4.50', 'Don:Neg-4.50', 'Don:Neutral-4.50',
       'Don:Pos-4.50', 'Don:Sul-4.50', 'Hydro:Hydro-4.50',
       'Hydro:Neg-4.50', 'Hydro:Neutral-4.50', 'Hydro:Pos-4.50',
       'Hydro:Sul-4.50', 'Neg:Neg-4.50', 'Neg:Neutral-4.50',
       'Neg:Pos-4.50', 'Neg:Sul-4.50', 'Neutral:Neutral-4.50',
       'Neutral:Pos-4.50', 'Neutral:Sul-4.50', 'Pos:Pos-4.50',
       'Pos:Sul-4.50', 'Sul:Sul-4.50', 'Acc:Acc-4.00', 'Acc:Aro-4.00',
       'Acc:Don-4.00', 'Acc:Hydro-4.00', 'Acc:Neg-4.00',
       'Acc:Neutral-4.00', 'Acc:Pos-4.00', 'Acc:Sul-4.00', 'Aro:Aro-4.00',
       'Aro:Don-4.00', 'Aro:Hydro-4.00', 'Aro:Neg-4.00',
       'Aro:Neutral-4.00', 'Aro:Pos-4.00', 'Aro:Sul-4.00', 'Don:Don-4.00',
       'Don:Hydro-4.00', 'Don:Neg-4.00', 'Don:Neutral-4.00',
       'Don:Pos-4.00', 'Don:Sul-4.00', 'Hydro:Hydro-4.00',
       'Hydro:Neg-4.00', 'Hydro:Neutral-4.00', 'Hydro:Pos-4.00',
       'Hydro:Sul-4.00', 'Neg:Neg-4.00', 'Neg:Neutral-4.00',
       'Neg:Pos-4.00', 'Neg:Sul-4.00', 'Neutral:Neutral-4.00',
       'Neutral:Pos-4.00', 'Neutral:Sul-4.00', 'Pos:Pos-4.00',
       'Pos:Sul-4.00', 'Sul:Sul-4.00', 'Acc:Acc-3.50', 'Acc:Aro-3.50',
       'Acc:Don-3.50', 'Acc:Hydro-3.50', 'Acc:Neg-3.50',
       'Acc:Neutral-3.50', 'Acc:Pos-3.50', 'Acc:Sul-3.50', 'Aro:Aro-3.50',
       'Aro:Don-3.50', 'Aro:Hydro-3.50', 'Aro:Neg-3.50',
       'Aro:Neutral-3.50', 'Aro:Pos-3.50', 'Aro:Sul-3.50', 'Don:Don-3.50',
       'Don:Hydro-3.50', 'Don:Neg-3.50', 'Don:Neutral-3.50',
       'Don:Pos-3.50', 'Don:Sul-3.50', 'Hydro:Hydro-3.50',
       'Hydro:Neg-3.50', 'Hydro:Neutral-3.50', 'Hydro:Pos-3.50',
       'Hydro:Sul-3.50', 'Neg:Neg-3.50', 'Neg:Neutral-3.50',
       'Neg:Pos-3.50', 'Neg:Sul-3.50', 'Neutral:Neutral-3.50',
       'Neutral:Pos-3.50', 'Neutral:Sul-3.50', 'Pos:Pos-3.50',
       'Pos:Sul-3.50', 'Sul:Sul-3.50', 'Acc:Acc-3.00', 'Acc:Aro-3.00',
       'Acc:Don-3.00', 'Acc:Hydro-3.00', 'Acc:Neg-3.00',
       'Acc:Neutral-3.00', 'Acc:Pos-3.00', 'Acc:Sul-3.00', 'Aro:Aro-3.00',
       'Aro:Don-3.00', 'Aro:Hydro-3.00', 'Aro:Neg-3.00',
       'Aro:Neutral-3.00', 'Aro:Pos-3.00', 'Aro:Sul-3.00', 'Don:Don-3.00',
       'Don:Hydro-3.00', 'Don:Neg-3.00', 'Don:Neutral-3.00',
       'Don:Pos-3.00', 'Don:Sul-3.00', 'Hydro:Hydro-3.00',
       'Hydro:Neg-3.00', 'Hydro:Neutral-3.00', 'Hydro:Pos-3.00',
       'Hydro:Sul-3.00', 'Neg:Neg-3.00', 'Neg:Neutral-3.00',
       'Neg:Pos-3.00', 'Neg:Sul-3.00', 'Neutral:Neutral-3.00',
       'Neutral:Pos-3.00', 'Neutral:Sul-3.00', 'Pos:Pos-3.00',
       'Pos:Sul-3.00', 'Sul:Sul-3.00', 'Acc:Acc-2.50', 'Acc:Aro-2.50',
       'Acc:Don-2.50', 'Acc:Hydro-2.50', 'Acc:Neg-2.50',
       'Acc:Neutral-2.50', 'Acc:Pos-2.50', 'Acc:Sul-2.50', 'Aro:Aro-2.50',
       'Aro:Don-2.50', 'Aro:Hydro-2.50', 'Aro:Neg-2.50',
       'Aro:Neutral-2.50', 'Aro:Pos-2.50', 'Aro:Sul-2.50', 'Don:Don-2.50',
       'Don:Hydro-2.50', 'Don:Neg-2.50', 'Don:Neutral-2.50',
       'Don:Pos-2.50', 'Don:Sul-2.50', 'Hydro:Hydro-2.50',
       'Hydro:Neg-2.50', 'Hydro:Neutral-2.50', 'Hydro:Pos-2.50',
       'Hydro:Sul-2.50', 'Neg:Neg-2.50', 'Neg:Neutral-2.50',
       'Neg:Pos-2.50', 'Neg:Sul-2.50', 'Neutral:Neutral-2.50',
       'Neutral:Pos-2.50', 'Neutral:Sul-2.50', 'Pos:Pos-2.50',
       'Pos:Sul-2.50', 'Sul:Sul-2.50', 'Acc:Acc-2.00', 'Acc:Aro-2.00',
       'Acc:Don-2.00', 'Acc:Hydro-2.00', 'Acc:Neg-2.00',
       'Acc:Neutral-2.00', 'Acc:Pos-2.00', 'Acc:Sul-2.00', 'Aro:Aro-2.00',
       'Aro:Don-2.00', 'Aro:Hydro-2.00', 'Aro:Neg-2.00',
       'Aro:Neutral-2.00', 'Aro:Pos-2.00', 'Aro:Sul-2.00', 'Don:Don-2.00',
       'Don:Hydro-2.00', 'Don:Neg-2.00', 'Don:Neutral-2.00',
       'Don:Pos-2.00', 'Don:Sul-2.00', 'Hydro:Hydro-2.00',
       'Hydro:Neg-2.00', 'Hydro:Neutral-2.00', 'Hydro:Pos-2.00',
       'Hydro:Sul-2.00', 'Neg:Neg-2.00', 'Neg:Neutral-2.00',
       'Neg:Pos-2.00', 'Neg:Sul-2.00', 'Neutral:Neutral-2.00',
       'Neutral:Pos-2.00', 'Neutral:Sul-2.00', 'Pos:Pos-2.00',
       'Pos:Sul-2.00', 'Sul:Sul-2.00', 'Acc:Acc-1.50', 'Acc:Aro-1.50',
       'Acc:Don-1.50', 'Acc:Hydro-1.50', 'Acc:Neg-1.50',
       'Acc:Neutral-1.50', 'Acc:Pos-1.50', 'Acc:Sul-1.50', 'Aro:Aro-1.50',
       'Aro:Don-1.50', 'Aro:Hydro-1.50', 'Aro:Neg-1.50',
       'Aro:Neutral-1.50', 'Aro:Pos-1.50', 'Aro:Sul-1.50', 'Don:Don-1.50',
       'Don:Hydro-1.50', 'Don:Neg-1.50', 'Don:Neutral-1.50',
       'Don:Pos-1.50', 'Don:Sul-1.50', 'Hydro:Hydro-1.50',
       'Hydro:Neg-1.50', 'Hydro:Neutral-1.50', 'Hydro:Pos-1.50',
       'Hydro:Sul-1.50', 'Neg:Neg-1.50', 'Neg:Neutral-1.50',
       'Neg:Pos-1.50', 'Neg:Sul-1.50', 'Neutral:Neutral-1.50',
       'Neutral:Pos-1.50', 'Neutral:Sul-1.50', 'Pos:Pos-1.50',
       'Pos:Sul-1.50', 'Sul:Sul-1.50', 'Acc:Acc-1.00', 'Acc:Aro-1.00',
       'Acc:Don-1.00', 'Acc:Hydro-1.00', 'Acc:Neg-1.00',
       'Acc:Neutral-1.00', 'Acc:Pos-1.00', 'Acc:Sul-1.00', 'Aro:Aro-1.00',
       'Aro:Don-1.00', 'Aro:Hydro-1.00', 'Aro:Neg-1.00',
       'Aro:Neutral-1.00', 'Aro:Pos-1.00', 'Aro:Sul-1.00', 'Don:Don-1.00',
       'Don:Hydro-1.00', 'Don:Neg-1.00', 'Don:Neutral-1.00',
       'Don:Pos-1.00', 'Don:Sul-1.00', 'Hydro:Hydro-1.00',
       'Hydro:Neg-1.00', 'Hydro:Neutral-1.00', 'Hydro:Pos-1.00',
       'Hydro:Sul-1.00', 'Neg:Neg-1.00', 'Neg:Neutral-1.00',
       'Neg:Pos-1.00', 'Neg:Sul-1.00', 'Neutral:Neutral-1.00',
       'Neutral:Pos-1.00', 'Neutral:Sul-1.00', 'Pos:Pos-1.00',
       'Pos:Sul-1.00', 'Sul:Sul-1.00'])

seq_feat = np.array(['ALTS910101', 'AZAE970101', 'AZAE970102', 'BASU010101',
       'BENS940101', 'BENS940102', 'BENS940103', 'BENS940104',
       'BETM990101', 'BLAJ010101', 'BONM030101', 'BONM030102',
       'BONM030103', 'BONM030104', 'BONM030105', 'BONM030106',
       'BRYS930101', 'CROG050101', 'CSEM940101', 'DAYM780301',
       'DAYM780302', 'DOSZ010101', 'DOSZ010102', 'DOSZ010103',
       'DOSZ010104', 'FEND850101', 'FITW660101', 'GEOD900101',
       'GIAG010101', 'GODA950101', 'GONG920101', 'GRAR740104',
       'HENS920101', 'HENS920102', 'HENS920103', 'HENS920104',
       'JOHM930101', 'JOND920103', 'JOND940101', 'KANM000101',
       'KAPO950101', 'KESO980101', 'KESO980102', 'KOLA920101',
       'KOLA930101', 'KOSJ950100_RSA_SST', 'KOSJ950100_SST',
       'KOSJ950110_RSA', 'KOSJ950115', 'LEVJ860101', 'LINK010101',
       'LIWA970101', 'LUTR910101', 'LUTR910102', 'LUTR910103',
       'LUTR910104', 'LUTR910105', 'LUTR910106', 'LUTR910107',
       'LUTR910108', 'LUTR910109', 'MCLA710101', 'MCLA720101',
       'MEHP950101', 'MEHP950102', 'MEHP950103', 'MICC010101',
       'MIRL960101', 'MIYS850102', 'MIYS850103', 'MIYS930101',
       'MIYS960101', 'MIYS960102', 'MIYS960103', 'MIYS990106',
       'MIYS990107', 'MIYT790101', 'MOHR870101', 'MOOG990101',
       'MUET010101', 'MUET020101', 'MUET020102', 'NAOD960101',
       'NGPC000101', 'NIEK910101', 'NIEK910102', 'OGAK980101',
       'OVEJ920100_RSA', 'OVEJ920101', 'OVEJ920102', 'OVEJ920103',
       'PARB960101', 'PARB960102', 'PRLA000101', 'PRLA000102',
       'QUIB020101', 'QU_C930101', 'QU_C930102', 'QU_C930103',
       'RIER950101', 'RISJ880101', 'ROBB790102', 'RUSR970101',
       'RUSR970102', 'RUSR970103', 'SIMK990101', 'SIMK990102',
       'SIMK990103', 'SIMK990104', 'SIMK990105', 'SKOJ000101',
       'SKOJ000102', 'SKOJ970101', 'TANS760101', 'TANS760102',
       'THOP960101', 'TOBD000101', 'TOBD000102', 'TUDE900101',
       'VENM980101', 'VOGG950101', 'WEIL970101', 'WEIL970102',
       'ZHAC000101', 'ZHAC000102', 'ZHAC000103', 'ZHAC000104',
       'ZHAC000105', 'ZHAC000106', 'blosum100', 'blosum30', 'blosum35',
       'blosum40', 'blosum45', 'blosum50', 'blosum55', 'blosum60',
       'blosum62', 'blosum65', 'blosum70', 'blosum75', 'blosum80',
       'blosum85', 'blosum90', 'blosum95', 'pam120', 'pam180', 'pam250',
       'pam30', 'pam300', 'pam60', 'pam90'])

str_feat = np.array(['RSA', 'SST', 'Phi', 'Psi',
       'ResDepth', 'ca_depth', 'relative_bfactor', 'vol_change',
       'Covalent', 'VDWClash', 'VDW', 'Proximal', 'Hbond', 'WeakHbond',
       'Ionic', 'Aromatic', 'Hydrophobic', 'Carbonyl', 'Polar',
       'WeakPolar', 'PI-PI', 'Carbon-PI', 'Cation-PI', 'Donor-PI',
       'MetalSulphur-PI', 'Amide-Amide', 'Amide-Ring', 'd_Covalent',
       'd_VDWClash', 'd_VDW', 'd_Proximal', 'd_Hbond', 'd_WeakHbond',
       'd_Ionic', 'd_Aromatic', 'd_Hydrophobic', 'd_Carbonyl', 'd_Polar',
       'd_WeakPolar', 'd_PI-PI', 'd_Carbon-PI', 'd_Cation-PI',
       'd_Donor-PI', 'd_MetalSulphur-PI', 'd_Amide-Amide', 'd_Amide-Ring',
       'Hydro', 'Pos', 'Neg', 'Acc', 'Don', 'Aro', 'Sul', 'Neut'])

In [78]:
#str_feat -> replace SST with other features (from categorical encoding)
str_feat = np.delete(str_feat, np.where(str_feat == 'SST'))
new_elements = np.array(['SST_3-10 Helix', 'SST_Alpha Helix', 'SST_Bend', 
                         'SST_Isolated beta-bridge', 'SST_Pi Helix', 'SST_Strand', 
                         'SST_Turn'])
# Add new elements to the list
str_feat = np.append(str_feat, new_elements)

In [79]:
#Save numpy array for list of feature as csv
feat_list_path = '/home/korawich/Desktop/AutoML/Dataset/DDMut_data/Feat_list/'
gbsig_feat_list_path = feat_list_path + 'gbsig_feat_list.csv'
seq_feat_list_path = feat_list_path + 'seq_feat_list.csv'
str_feat_list_path = feat_list_path + 'str_feat_list.csv'

pd.DataFrame(gbsig_feat).to_csv(gbsig_feat_list_path)
pd.DataFrame(seq_feat).to_csv(seq_feat_list_path)
pd.DataFrame(str_feat).to_csv(str_feat_list_path)

Imputating missing values & Categorical encoding

In [56]:
import pandas as pd
import numpy as np
import time
import sys
import os
import argparse
import joblib
import re

In [57]:
data_path = '/home/korawich/Desktop/AutoML/Dataset/DDMut_data/'
gbsig_path = '/home/korawich/Desktop/AutoML/Dataset/DDMut_data/Graph-based/'
final_path = '/home/korawich/Desktop/AutoML/Dataset/DDMut_data/Final/'

In [58]:
#Input
train_info_path  = final_path + 'train_info.csv'
test_1_info_path = final_path + 'test_1_info.csv'
test_2_info_path = final_path + 'test_2_info.csv'
test_3_info_path = final_path + 'test_3_info.csv'

train_X_path  = final_path + 'train_X.csv'
test_1_X_path = final_path + 'test_1_X.csv'
test_2_X_path = final_path + 'test_2_X.csv'
test_3_X_path = final_path + 'test_3_X.csv'

train_y_path  = final_path + 'train_y.csv'
test_1_y_path = final_path + 'test_1_y.csv'
test_2_y_path = final_path + 'test_2_y.csv'
test_3_y_path = final_path + 'test_3_y.csv'

train_y_cls_path  = final_path + 'train_y_cls.csv'
test_1_y_cls_path = final_path + 'test_1_y_cls.csv'
test_2_y_cls_path = final_path + 'test_2_y_cls.csv'
test_3_y_cls_path = final_path + 'test_3_y_cls.csv'

In [59]:
X_train = pd.read_csv(train_X_path, index_col=0)
X_test_1 = pd.read_csv(test_1_X_path, index_col=0)
X_test_2 = pd.read_csv(test_2_X_path, index_col=0)
X_test_3 = pd.read_csv(test_3_X_path, index_col=0)

y_train = pd.read_csv(train_y_path, index_col=0)
y_test_1 = pd.read_csv(test_1_y_path, index_col=0)
y_test_2 = pd.read_csv(test_2_y_path, index_col=0)
y_test_3 = pd.read_csv(test_3_y_path, index_col=0)

/tmp/ipykernel_28381/297724681.py:3: DtypeWarning: Columns (77) have mixed types. Specify dtype option on import or set low_memory=False.
  X_test_2 = pd.read_csv(test_2_X_path, index_col=0)


In [60]:
#Check column with object type (contain '-')
object_type_columns = X_train.select_dtypes(include=['object']).columns.tolist()

# Print the columns with object type
print("Columns with object type:", object_type_columns)

Columns with object type: ['MEHP950101', 'MEHP950103', 'SST']


In [62]:
#Imputate missing values in one column :)
#1 - 2 column ('-')
from sklearn.impute import SimpleImputer

for column in ['MEHP950101', 'MEHP950103']:
    X_train[column]  = X_train[column].replace('-', np.nan).astype(float)
    X_test_1[column] = X_test_1[column].replace('-', np.nan).astype(float)
    X_test_2[column] = X_test_2[column].replace('-', np.nan).astype(float)
    X_test_3[column] = X_test_3[column].replace('-', np.nan).astype(float)
    
    imp = SimpleImputer(missing_values=np.nan, strategy='mean')
    X_train[column] = imp.fit_transform(X_train[[column]]).astype(float)
    X_test_1[column] = imp.transform(X_test_1[[column]]).astype(float)
    X_test_2[column] = imp.transform(X_test_2[[column]]).astype(float)
    X_test_3[column] = imp.transform(X_test_3[[column]]).astype(float)

#3, 4, 5 columns (NaN)
for column in ['GODA950101', 'PARB960101', 'PARB960102', 'ROBB790102']:
    imp = SimpleImputer(missing_values=np.nan, strategy='mean')
    X_train[column] = imp.fit_transform(X_train[[column]]).astype(float)
    X_test_1[column] = imp.transform(X_test_1[[column]]).astype(float)
    X_test_2[column] = imp.transform(X_test_2[[column]]).astype(float)
    X_test_3[column] = imp.transform(X_test_3[[column]]).astype(float)

In [63]:
# Get columns with NaN  (Check that there is no NaN anymore)
nan_columns = X_train.columns[X_train.isna().any()].tolist()

# Print the list of columns with NaN values
print("Columns with NaN values:", nan_columns)

Columns with NaN values: ['SST']


In [64]:
#Categorical encoding
#1) Checking
print('variables in "SST" columns : ', np.unique(X_train['SST'].astype(str).values))
print('variables in "SST" columns : ', np.unique(X_test_1['SST'].astype(str).values))
print('variables in "SST" columns : ', np.unique(X_test_2['SST'].astype(str).values))
print('variables in "SST" columns : ', np.unique(X_test_3['SST'].astype(str).values))


#2) Encoding
X_train = pd.get_dummies(X_train, columns=['SST'])
X_test_1 = pd.get_dummies(X_test_1, columns=['SST'])
X_test_2 = pd.get_dummies(X_test_2, columns=['SST'])
X_test_3 = pd.get_dummies(X_test_3, columns=['SST'])

for i in ['SST_3-10 Helix', 'SST_Alpha Helix', 'SST_Bend', 'SST_Isolated beta-bridge', 'SST_Pi Helix', 'SST_Strand', 'SST_Turn']:
    for j in [X_train, X_test_1, X_test_2, X_test_3]:
        if not i in j:
            j[i] = False

variables in "SST" columns :  ['3-10 Helix' 'Alpha Helix' 'Bend' 'Isolated beta-bridge' 'Pi Helix'
 'Strand' 'Turn' 'nan']
variables in "SST" columns :  ['3-10 Helix' 'Alpha Helix' 'Bend' 'Isolated beta-bridge' 'Strand' 'Turn'
 'nan']
variables in "SST" columns :  ['3-10 Helix' 'Alpha Helix' 'Bend' 'Isolated beta-bridge' 'Pi Helix'
 'Strand' 'Turn' 'nan']
variables in "SST" columns :  ['3-10 Helix' 'Alpha Helix' 'Bend' 'Isolated beta-bridge' 'Pi Helix'
 'Strand' 'Turn' 'nan']


In [65]:
#3) Check if they have the same number of features
print('X_train.shape = ', X_train.shape)
print('X_test_1.shape = ', X_test_1.shape)
print('X_test_2.shape = ', X_test_2.shape)
print('X_test_3.shape = ', X_test_3.shape)

X_train.shape =  (9028, 896)
X_test_1.shape =  (552, 896)
X_test_2.shape =  (1304, 896)
X_test_3.shape =  (2024, 896)


In [66]:
#Reindex to match them
X_test_1 = X_test_1.reindex(columns=X_train.columns, fill_value=0)
X_test_2 = X_test_2.reindex(columns=X_train.columns, fill_value=0)
X_test_3 = X_test_3.reindex(columns=X_train.columns, fill_value=0)

In [67]:
#Output
train_X_path  = final_path + 'final_train_X.csv'
test_1_X_path = final_path + 'final_test_1_X.csv'
test_2_X_path = final_path + 'final_test_2_X.csv'
test_3_X_path = final_path + 'final_test_3_X.csv'

In [68]:
X_train.to_csv(train_X_path)
X_test_1.to_csv(test_1_X_path)
X_test_2.to_csv(test_2_X_path)
X_test_3.to_csv(test_3_X_path)